### Импорт

In [ ]:
!pip install pymorphy2
!python3 -m spacy download en_core_web_sm

In [ ]:
import re
import json
import nltk
import spacy
import string
import pymorphy2
import pandas as pd
import numpy as np
import gensim.models
import matplotlib.pyplot as plt

from zipfile import ZipFile
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer

random_state=9

In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

In [ ]:
api_token = {"username":"w1nston","key":"81bfe28fc3d2a7574476ab1222e695db"}

with open('/root/.kaggle/kaggle.json', 'w+') as file:
    json.dump(api_token, file)

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ozlerhakan/spam-or-not-spam-dataset

In [ ]:
with ZipFile('spam-or-not-spam-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall()

### Предобработка

In [111]:
data = pd.read_csv('spam_or_not_spam.csv', encoding='iso-8859-1').rename(columns={'email': 'text'})

In [ ]:
data = data[['text', 'label']].dropna()

In [113]:
data['cleaned_text'] = data['text'].apply(
    lambda x: ' '.join(
        token.lemma_.lower() for token in nlp(x) if
        not token.is_stop
        and not token.is_punct
        and not token.is_digit
        and not token.like_email
        and not token.like_num
        and not token.is_space
    )
)

In [115]:
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['label'], random_state=random_state)

In [126]:
sentences = [elem.split() for elem in data['cleaned_text']]

### SG, CBOW, FastText сравнение

In [135]:
# SG
sg_model = gensim.models.Word2Vec(
    sentences=sentences,
    vector_size=256, # default = 100
    window=7, # default = 5
    min_count=10,
    sg=1,
    hs=0, #  If 1, hierarchical softmax will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
    negative=5, # If > 0, negative sampling will be used, if set to 0, no negative sampling is used.
    epochs=25, # Number of iterations (epochs) over the corpus
    seed=random_state,
)

In [137]:
# CBOW

cbow_model = gensim.models.Word2Vec(
    sentences=sentences,
    vector_size=256, # default = 100
    window=7, # default = 5
    min_count=10,
    sg=0, # Training algorithm: 1 for skip-gram; otherwise CBOW
    hs=0, #  If 1, hierarchical softmax will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
    negative=5, # If > 0, negative sampling will be used, if set to 0, no negative sampling is used.
    epochs=25, # Number of iterations (epochs) over the corpus
    seed=random_state,
)

In [136]:
# FastText
gensim.models.fasttext.FastText
ft_model = gensim.models.Word2Vec(
    sentences=sentences,
    vector_size=256, # default = 100
    window=7, # default = 5
    min_count=10,
    sg=0, # Training algorithm: 1 for skip-gram; otherwise CBOW
    hs=0, #  If 1, hierarchical softmax will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
    negative=5, # If > 0, negative sampling will be used, if set to 0, no negative sampling is used.
    epochs=25, # Number of iterations (epochs) over the corpus
    seed=random_state,
)

In [141]:
sg_model.wv.most_similar(positive=['tree'], topn=5)

[('synaptic', 0.4120144248008728),
 ('dependency', 0.3835029602050781),
 ('canarsie', 0.3753243386745453),
 ('reading', 0.3733203709125519),
 ('insect', 0.3562522828578949)]

In [142]:
cbow_model.wv.most_similar(positive=['tree'], topn=5)

[('dependency', 0.6039867997169495),
 ('mpnumber', 0.4963988661766052),
 ('synaptic', 0.4784884452819824),
 ('xmms', 0.4763685166835785),
 ('package', 0.47485804557800293)]

In [143]:
ft_model.wv.most_similar(positive=['tree'], topn=5)

[('dependency', 0.5085680484771729),
 ('ton', 0.47516703605651855),
 ('object', 0.4613674581050873),
 ('tip', 0.4573879539966583),
 ('package', 0.44909632205963135)]

In [147]:
print(sg_model.wv.doesnt_match(['january', 'february', 'march', 'tree']))
print(cbow_model.wv.doesnt_match(['january', 'february', 'march', 'tree']))
print(ft_model.wv.doesnt_match(['january', 'february', 'march', 'tree']))

tree
tree
tree


### Обучение LogisticRegression

пока не получилось, но я пытаюсь

In [ ]:
mapping = LogisticRegression(fit_intercept=False, random_state=random_state)
mapping.fit(X_train, y_train)
preds = mapping.predict(X_test_vectorized)
print(classification_report(y_test, preds))

In [ ]:
logreg = LogisticRegression().fit(X_train_vectorized, y_train)
preds = logreg.predict(X_test_vectorized)
print(classification_report(y_test, preds))